## 과제 설명
법률 문서에 대한 원문을 가장 잘 나타내는 3개의 문장을 추출하는 문서 요약 모델 개발
    
### 데이터 설명
- 개요 : 뉴스 기사, 기고문, 잡지, 법률 (판결문) 등 다양한 영역에서 추출된 텍스트 데이터와 요약본 40만 건
- 입출력: 
    - 입력: 문장별로 나뉜 법률 문서 원문, 예) 법률 문서 = [문장1, 문장2, ..., 문장K], K : 문서 길이
    - 출력: 요약문에 포함될 문장 인덱스 3개
- 데이터셋 구성
    - 학습 데이터:
        - train.json : 24,027개의 법률 문서 아이디 (id), 원문 (article_original), 요약문 (extractive)
    - 테스트 데이터:
        - test.json : 3,004개의 법률 문서 아이디 (id), 원문 (article_original)

### 사용 pretrained 모델
 `beomi/KcELECTRA-base` 
[Documentation](https://github.com/Beomi/KcELECTRA)


## 세팅
### 라이브러리


In [1]:
# 설치되지 않은 라이브러리의 경우, 주석 해제 후 코드를 실행하여 설치
# !pip install pytorch-pretrained-bert
# !pip install transformers

In [2]:
import os
# os.mkdir("./data")

In [3]:
# 필요한 라이브러리 및 코드 파일 불러오기

import time
import torch
import torch.optim as optim
from torch.utils.data import DataLoader

from datetime import datetime, timezone, timedelta
import numpy as np
import random


In [4]:
# 시드 고정
RANDOM_SEED = 42
torch.manual_seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

# Set device
os.environ["CUDA_VISIBLE_DEVICES"]="0"
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 경로 설정
ROOT_PATH = './'
DATA_DIR = './data'
MODEL_DIR = DATA_DIR


## 데이터 로드

In [5]:
# hyper-parameters
TRAIN_BATCH_SIZE = 8
EVAL_BATCH_SIZE = 32

# 학습 데이터만 있으니 학습 데이터셋 비율과 validation 데이터셋 비율을 나눔
TRAIN_RATIO = 0.9

- `__init__` 에서 tokenizer는 transformers 라이브러리에서 AutoTokenizer를 사용합니다. 이 외에도 원하는 토크나이저를 적용해 다양한 실험을 진행할 수 있습니다.

In [6]:
import pandas as pd
from torch.utils.data import Dataset
from pytorch_pretrained_bert import BertTokenizer
from transformers import AutoTokenizer
from itertools import chain
import json

class CustomDataset(Dataset):
    def __init__(self, data_dir, mode):
        self.data_dir = data_dir
        self.mode = mode
        self.tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base")
        self.inputs, self.labels = self.data_loader()

    def data_loader(self):
        print('Loading ' + self.mode + ' dataset..')
        
        if os.path.isfile(os.path.join(self.data_dir, self.mode + '_X.pt')):
            inputs = torch.load(os.path.join(self.data_dir, self.mode + '_X.pt'))
            labels = torch.load(os.path.join(self.data_dir, self.mode + '_Y.pt'))

        else:
            file_path = os.path.join(self.data_dir, 'train.json')
            df = pd.read_json(file_path, orient='records', encoding='utf-8-sig')
          
            if self.mode == 'train':
                df = df.loc[:TRAIN_RATIO*int(len(df)), :]
            elif self.mode == 'val':
                df = df.loc[TRAIN_RATIO*int(len(df)):, :]

            inputs = pd.DataFrame(columns=['src'])
            labels = pd.DataFrame(columns=['trg'])
            inputs['src'] =  df['article_original']
            labels['trg'] =  df['extractive']
          
            # Preprocessing
            inputs, labels = self.preprocessing(inputs, labels)
            # Save data
            torch.save(inputs ,os.path.join(self.data_dir, self.mode + '_X.pt'))
            torch.save(labels, os.path.join(self.data_dir, self.mode + '_Y.pt'))

        inputs = inputs.values
        labels = labels.values

        return inputs, labels

    def pad(self, data, pad_id, max_len):
        padded_data = data.map(lambda x : torch.cat([x, torch.tensor([pad_id] * (max_len - len(x)))]))
        return padded_data

    def preprocessing(self, inputs, labels):
        print('Preprocessing ' + self.mode + ' dataset..')
        #Encoding original text
        inputs['src'] = inputs['src'].map(lambda x: torch.tensor(list(chain.from_iterable([self.tokenizer.encode(x[i], max_length = int(512 / len(x)),  add_special_tokens=True) for i in range(len(x))]))))
        inputs['clss'] = inputs.src.map(lambda x : torch.cat([torch.where(x == 2)[0], torch.tensor([len(x)])]))
        inputs['segs'] = inputs.clss.map(lambda x : torch.tensor(list(chain.from_iterable([[0] * (x[i+1] - x[i]) if i % 2 == 0 else [1] * (x[i+1] - x[i]) for i, val in enumerate(x[:-1])]))))
        inputs['clss'] = inputs.clss.map(lambda x : x[:-1])

        ##Padding
        max_encoding_len = max(inputs.src.map(lambda x: len(x)))
        max_label_len = max(inputs.clss.map(lambda x: len(x)))
        inputs['src'] = self.pad(inputs.src, 0, max_encoding_len)
        inputs['segs'] = self.pad(inputs.segs, 0, max_encoding_len)
        inputs['clss'] = self.pad(inputs.clss, -1, max_label_len)
        inputs['mask'] = inputs.src.map(lambda x: ~ (x == 0))
        inputs['mask_clss'] = inputs.clss.map(lambda x: ~ (x == -1))

        #Binarize label {Extracted sentence : 1, Not Extracted sentence : 0}
        labels = labels['trg'].map(lambda  x: torch.tensor([1 if i in x else 0 for i in range(max_label_len)]))

        return inputs, labels

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, index):
        return [self.inputs[index][i] for i in range(5)], self.labels[index]


In [7]:
# Load dataset & dataloader
train_dataset = CustomDataset(data_dir=DATA_DIR, mode='train')
validation_dataset = CustomDataset(data_dir=DATA_DIR, mode='val')

train_dataloader = DataLoader(dataset=train_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True)
validation_dataloader = DataLoader(dataset=validation_dataset, batch_size=EVAL_BATCH_SIZE, shuffle=False)

Loading train dataset..
Loading val dataset..


## 모델설계

In [8]:
# parameters
EPOCHS = 5
LEARNING_RATE = 0.0005
WEIGHT_DECAY = 0.00001
NUM_WORKERS = 0
EARLY_STOPPING_PATIENCE = 2

- 한국어 자연어 처리의 pretrained model인 KcELECTRA 깃헙 [https://github.com/Beomi/KcELECTRA] 참고
- !주의! 모델이 무거우니 사용하는 파라미터 개수와 개발 환경 등을 고려하여 인코더 등을 선택하세요

In [17]:
from torch import nn
import transformers
from sklearn.metrics import f1_score

class Summarizer(nn.Module):

    def __init__(self):
        """
        """
        super(Summarizer, self).__init__()
        self.encoder = transformers.DistilBertModel.from_pretrained('distilbert-base-multilingual-cased')
        self.fc = nn.Linear(768, 1)
        self.sigmoid = nn.Sigmoid()


    def forward(self, x, segs, clss, mask, mask_clss, sentence_range=None):
        """
        """
        top_vec = self.encoder(input_ids = x.long(), attention_mask = mask.float()).last_hidden_state
        sents_vec = top_vec[torch.arange(top_vec.size(0)).unsqueeze(1), clss.long()]
        sents_vec = sents_vec * mask_clss[:, :, None].float()
        h = self.fc(sents_vec).squeeze(-1)
        sent_scores = self.sigmoid(h) * mask_clss.float()
        return sent_scores
    

In [18]:
model = Summarizer().to(DEVICE)

Some weights of the model checkpoint at distilbert-base-multilingual-cased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [19]:
def Hitrate(y_true, y_pred):
    """ Metric 함수 반환하는 함수

    Returns:
        metric_fn (Callable)
    """
    hitrate = np.array([len(list(set(ans).intersection(y_true[i])))/3 for i, ans in enumerate(y_pred)])
    score = np.mean(hitrate)
    return score

In [20]:
# Set optimizer, scheduler, loss function, metric function
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e5, max_lr=0.0001, epochs=EPOCHS, steps_per_epoch=len(train_dataloader))
loss_fn = torch.nn.BCELoss(reduction='none')

# Set metrics
metric_fn = Hitrate

In [21]:
import logging
class LossEarlyStopper():
    """Early stopper
    
    Attributes:
        patience (int): loss가 줄어들지 않아도 학습할 epoch 수
        verbose (bool): 로그 출력 여부, True 일 때 로그 출력
        patience_counter (int): loss 가 줄어들지 않을 때 마다 1씩 증가
        min_loss (float): 최소 loss
        stop (bool): True 일 때 학습 중단

    """

    def __init__(self, patience: int, verbose: bool, logger:logging.RootLogger=None)-> None:
        """ 초기화

        Args:
            patience (int): loss가 줄어들지 않아도 학습할 epoch 수
            weight_path (str): weight 저장경로
            verbose (bool): 로그 출력 여부, True 일 때 로그 출력
        """
        self.patience = patience
        self.verbose = verbose

        self.patience_counter = 0
        self.min_loss = np.Inf
        self.logger = logger
        self.stop = False

    def check_early_stopping(self, loss: float)-> None:
        """Early stopping 여부 판단

        Args:
            loss (float):

        Examples:
            
        Note:
            
        """  

        if self.min_loss == np.Inf:
            self.min_loss = loss
            # self.save_checkpoint(loss=loss, model=model)

        elif loss > self.min_loss:
            self.patience_counter += 1
            msg = f"Early stopper, Early stopping counter {self.patience_counter}/{self.patience}"

            if self.patience_counter == self.patience:
                self.stop = True

            if self.verbose:
                self.logger.info(msg) if self.logger else print(msg)
                
        elif loss <= self.min_loss:
            self.save_model = True
            msg = f"Early stopper, Validation loss decreased {self.min_loss} -> {loss}"
            self.min_loss = loss
            # self.save_checkpoint(loss=loss, model=model)

            if self.verbose:
                self.logger.info(msg) if self.logger else print(msg)


In [22]:
class Trainer():
    """ Trainer
        epoch에 대한 학습 및 검증 절차 정의
    
    Attributes:
        model (`model`)
        device (str)
        loss_fn (Callable)
        metric_fn (Callable)
        optimizer (`optimizer`)
        scheduler (`scheduler`)
    """

    def __init__(self, model, device, loss_fn, metric_fn, optimizer=None, scheduler=None, logger=None):
        """ 초기화
        """
        self.model = model
        self.device = device
        self.loss_fn = loss_fn
        self.metric_fn = metric_fn
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.logger = logger

    def train_epoch(self, dataloader, epoch_index):
        """ 한 epoch에서 수행되는 학습 절차

        Args:
            dataloader (`dataloader`)
            epoch_index (int)
        """
        self.model.train()
        self.train_total_loss = 0
        pred_lst = []
        target_lst = []
        for batch_index, (data, target) in enumerate(tqdm(dataloader)):
            self.optimizer.zero_grad()
            src = data[0].to(self.device)
            clss = data[1].to(self.device)
            segs = data[2].to(self.device)
            mask = data[3].to(self.device)
            mask_clss = data[4].to(self.device)
            target = target.float().to(self.device)
            sent_score = self.model(src, segs, clss, mask, mask_clss)
            loss = self.loss_fn(sent_score, target)
            loss = (loss * mask_clss.float()).sum()
            self.train_total_loss += loss
            loss.backward()
            self.optimizer.step()
            self.scheduler.step()
            pred_lst.extend(torch.topk(sent_score, 3, axis=1).indices.tolist())
            try:
                target_lst.extend(torch.where(target==1)[1].reshape(-1,3).tolist())
            except:
                print(target)
                sys.exit()
                
        self.train_mean_loss = self.train_total_loss / len(dataloader)
        self.train_score = self.metric_fn(y_true=target_lst, y_pred=pred_lst)
        msg = f'Epoch {epoch_index}, Train, loss: {self.train_mean_loss}, Score: {self.train_score}'
        print(msg)

    def validate_epoch(self, dataloader, epoch_index):
        """ 한 epoch에서 수행되는 검증 절차

        Args:
            dataloader (`dataloader`)
            epoch_index (int)
        """
        self.model.eval()
        self.val_total_loss = 0
        pred_lst = []
        target_lst = []

        with torch.no_grad():
            for batch_index, (data, target) in enumerate(tqdm(dataloader)):
                src = data[0].to(self.device)
                clss = data[1].to(self.device)
                segs = data[2].to(self.device)
                mask = data[3].to(self.device)
                mask_clss = data[4].to(self.device)
                target = target.float().to(self.device)
                sent_score = self.model(src, segs, clss, mask, mask_clss)
                loss = self.loss_fn(sent_score, target)
                loss = (loss * mask_clss.float()).sum()
                self.val_total_loss += loss
                pred_lst.extend(torch.topk(sent_score, 3, axis=1).indices.tolist())
                target_lst.extend(torch.where(target==1)[1].reshape(-1,3).tolist())
            self.val_mean_loss = self.val_total_loss / len(dataloader)
            self.validation_score = self.metric_fn(y_true=target_lst, y_pred=pred_lst)
            msg = f'Epoch {epoch_index}, Validation, loss: {self.val_mean_loss}, Score: {self.validation_score}'
            print(msg)


In [23]:
# Set trainer
trainer = Trainer(model, DEVICE, loss_fn, metric_fn, optimizer, scheduler)

# Set earlystopper
early_stopper = LossEarlyStopper(patience=EARLY_STOPPING_PATIENCE, verbose=True)

## 학습

In [24]:
# TRAIN
from tqdm import tqdm

start = time.time()
criterion = 0

for epoch_index in range(EPOCHS):
    
    trainer.train_epoch(train_dataloader, epoch_index=epoch_index)
    trainer.validate_epoch(validation_dataloader, epoch_index=epoch_index)
   
    # early_stopping check
    early_stopper.check_early_stopping(loss=trainer.val_mean_loss)

    if early_stopper.stop:
        print('Early stopped')
        break

    if trainer.validation_score > criterion:
        criterion = trainer.validation_score
        check_point = {
            'model' : model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'scheduler': scheduler.state_dict()
        }
        
        torch.save(check_point, os.path.join(MODEL_DIR, 'best.pt'))
        
        
print("train finished, best.pt saved.")

100%|██████████| 2704/2704 [11:16<00:00,  4.00it/s]


Epoch 0, Train, loss: 37.38749694824219, Score: 0.5852639691714836


100%|██████████| 76/76 [00:21<00:00,  3.61it/s]


Epoch 0, Validation, loss: 138.2972869873047, Score: 0.6236469608659451


100%|██████████| 2704/2704 [11:16<00:00,  4.00it/s]


...

Epoch 2, Validation, loss: 131.8885498046875, Score: 0.6514016097696363
Early stopper, Early stopping counter 1/2


100%|██████████| 2704/2704 [11:16<00:00,  4.00it/s]


Epoch 3, Train, loss: 27.50633430480957, Score: 0.7262273603082852


100%|██████████| 76/76 [00:21<00:00,  3.61it/s]

Epoch 3, Validation, loss: 144.5745086669922, Score: 0.631834582292534
Early stopper, Early stopping counter 2/2
Early stopped
train finished, best.pt saved.


## 추론
테스트 데이터의 타겟 변수를 `sample_submission.csv` 양식에 맞춰 저장하고, 해당 제출파일을 Inclass에 제출하시면 점수를 확인할 수 있습니다.

여러분의 모델의 추론 결과로 나온 문서 당 세 개의 요약 인덱스에 해당하는 "idx_#"을 1로 채워 제출 파일을 만듭니다(현재는 아래 보시는 바와 같이 모두 0으로 채워져 있습니다). ID 값을 기준으로 채점을 진행하는 점 유의해주시기 바랍니다.

In [26]:
submit = pd.read_csv(os.path.join(DATA_DIR,'sample_submission.csv'))
submit.head()

,ID,summary_index1,summary_index2,summary_index3
0,79095,0,1,2
1,204506,0,1,2
2,142079,0,1,2
3,110816,0,1,2
4,207249,0,1,2


In [27]:
# 테스트 데이터셋 클래스 정의

class TestDataset(Dataset):
    """ CustomDataset과 비슷한 구조이지만 레이블이 주어지지 않음을 염두 """

    def __init__(self, data_dir, mode):
        self.data_dir = data_dir
        self.mode = mode
        self.tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base")
        self.inputs = self.data_loader()

    def data_loader(self):
        print('Loading ' + self.mode + ' dataset..')
        if os.path.isfile(os.path.join(self.data_dir, self.mode+'_X.pt')):
            # torch tensor 불러오기
            inputs = torch.load(os.path.join(self.data_dir, self.mode + '_X.pt'))
        else:
            # json 파일 불러오기
            file_path = os.path.join(self.data_dir, self.mode + '.json')
            df = pd.read_json(file_path, orient='records', encoding='utf-8-sig')
            inputs = pd.DataFrame(columns=['src'])
            inputs['src'] =  df['article_original']
      
            # 전처리
            inputs = self.preprocessing(inputs)
            
            # 다음부터는 전처리 과정을 반복하지 않기 위해 tensor 저장
            torch.save(inputs ,os.path.join(self.data_dir, self.mode + '_X.pt'))

        inputs = inputs.values

        return inputs

    def pad(self, data, pad_id, max_len):
        padded_data = data.map(lambda x : torch.cat([x, torch.tensor([pad_id] * (max_len - len(x)))]))
        return padded_data

    def preprocessing(self, inputs):
        print('Preprocessing ' + self.mode + ' dataset..')
        
        #Encoding original text
        inputs['src'] = inputs['src'].map(lambda x: torch.tensor(list(chain.from_iterable([self.tokenizer.encode(x[i], max_length = int(512 / len(x)),  add_special_tokens=True) for i in range(len(x))]))))
        inputs['clss'] = inputs.src.map(lambda x : torch.cat([torch.where(x == 2)[0], torch.tensor([len(x)])]))
        inputs['segs'] = inputs.clss.map(lambda x : torch.tensor(list(chain.from_iterable([[0] * (x[i+1] - x[i]) if i % 2 == 0 else [1] * (x[i+1] - x[i]) for i, val in enumerate(x[:-1])]))))
        inputs['clss'] = inputs.clss.map(lambda x : x[:-1])

        ##Padding
        max_encoding_len = max(inputs.src.map(lambda x: len(x)))
        max_label_len = max(inputs.clss.map(lambda x: len(x)))
        inputs['src'] = self.pad(inputs.src, 0, max_encoding_len)
        inputs['segs'] = self.pad(inputs.segs, 0, max_encoding_len)
        inputs['clss'] = self.pad(inputs.clss, -1, max_label_len)
        inputs['mask'] = inputs.src.map(lambda x: ~ (x == 0))
        inputs['mask_clss'] = inputs.clss.map(lambda x: ~ (x == -1))
     
        return inputs

    def __len__(self):
        return len(self.inputs)
    
    def __getitem__(self, index):
        return [self.inputs[index][i] for i in range(5)]


In [28]:
# 테스트 데이터 로드
test_dataset = TestDataset(data_dir=DATA_DIR, mode = 'test')
test_dataloader = DataLoader(dataset=test_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=False)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Loading test dataset..
Preprocessing test dataset..


In [29]:
""" 이전에 학습한 모델 weight파일을 불러 추론하려면 아래 주석을 풀고 실행
    학습 진행 후 바로 추론하는 경우 학습 과정의 model 사용 (주석 풀지 않고 실행) """
MODEL_DIR = os.path.join(MODEL_DIR, 'best.pt')
model = Summarizer().to(DEVICE)
model.load_state_dict(torch.load(MODEL_DIR)['model'])

# 추론
model.eval()

# 추론 결과를 pred 리스트로 저장할 예정
pred_lst = []

with torch.no_grad():
    for batch_index, (data) in enumerate(test_dataloader):
        src = data[0].to(DEVICE)
        clss = data[1].to(DEVICE)
        segs = data[2].to(DEVICE)
        mask = data[3].to(DEVICE)
        mask_clss = data[4].to(DEVICE)
        sent_score = model(src, segs, clss, mask, mask_clss)
        pred_lst.extend(torch.topk(sent_score, 3, axis=1).indices.tolist())
            
        # 진행과정 출력
        if batch_index % 150 == 0:
            print(f'Prediction: {batch_index}/{len(test_dataloader)} completed')
    print("Prediction all completed")


Some weights of the model checkpoint at distilbert-base-multilingual-cased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Prediction: 0/376 completed
Prediction: 150/376 completed
Prediction: 300/376 completed
Prediction all completed


In [41]:
pred_lst

[[1, 5, 2],
 [9, 7, 0],
 [13, 8, 6],
 [9, 0, 7],
 [4, 3, 0],
 [0, 5, 4],
 [8, 7, 1],
 [5, 1, 4],
 [3, 4, 1],
 [3, 0, 1],
 [9, 5, 6],
 [1, 2, 4],
 [3, 2, 0],
 [3, 5, 2],
 [1, 0, 4],
 [4, 1, 3],
 [2, 3, 0],
 [10, 0, 9],
 [10, 0, 6],
 [0, 1, 6],
 [3, 0, 2],
 [16, 11, 15],
 [3, 1, 2],
 [8, 2, 0],
 [3, 0, 2],
 [4, 0, 5],
 [3, 0, 1],
 [12, 11, 1],
 [7, 9, 8],
 [0, 8, 5],
 [2, 4, 1],
 [5, 12, 11],
 [13, 28, 12],
 [0, 6, 5],
 [1, 3, 0],
 [4, 1, 0],
 [4, 0, 2],
 [4, 2, 0],
 [0, 2, 1],
 [18, 8, 3],
 [7, 4, 0],
 [15, 10, 12],
 [25, 0, 14],
 [4, 1, 3],
 [7, 12, 4],
 [4, 0, 1],
 [12, 6, 11],
 [20, 18, 17],
 [4, 0, 3],
 [12, 8, 7],
 [8, 3, 5],
 [3, 2, 0],
 [4, 2, 1],
 [14, 20, 16],
 [5, 3, 0],
 [0, 2, 1],
 [3, 2, 0],
 [4, 5, 2],
 [4, 1, 2],
 [6, 4, 0],
 [17, 8, 12],
 [0, 1, 3],
 [9, 7, 4],
 [5, 4, 3],
 [2, 3, 0],
 [3, 0, 1],
 [12, 13, 3],
 [8, 4, 3],
 [17, 11, 8],
 [4, 5, 3],
 [10, 1, 7],
 [5, 0, 2],
 [4, 5, 0],
 [5, 0, 4],
 [3, 6, 2],
 [12, 11, 8],
 [3, 0, 1],
 [3, 0, 2],
 [8, 7, 0],
 [3, 0, 1],
 [

In [45]:
# 제출 파일
for i,txt in enumerate(pred_lst):
    submit.iloc[i, 1] += txt[0]+1
    submit.iloc[i, 2] += txt[1]+1
    submit.iloc[i, 3] += txt[2]+1
    # print(submit.iloc[i, 1])
submit[submit["ID"] == 83319]

,ID,summary_index1,summary_index2,summary_index3
708,83319,60,121,117


In [46]:
submit.to_csv(os.path.join(ROOT_PATH, 'prediction.csv'), index=False)

In [47]:
predc = pd.read_csv("./prediction.csv")

In [49]:
predc[predc["ID"] == 83319]

,ID,summary_index1,summary_index2,summary_index3
708,83319,60,121,117
